In [45]:
import pandas as pd

file_in = "CC_Webb_Predictions_Prob_2_5sigFeb172023.csv"
csv = pd.read_csv(file_in)

csv.columns

Index(['Unnamed: 0', 'RA', 'DEC', 'size', 'isophotal_vegamag_f200w',
       'isophotal_vegamag_err_f200w', 'isophotal_vegamag_f090w',
       'isophotal_vegamag_err_f090w', 'isophotal_vegamag_f187n',
       'isophotal_vegamag_err_f187n', 'isophotal_vegamag_f335m',
       'isophotal_vegamag_err_f335m', 'isophotal_vegamag_f444w',
       'isophotal_vegamag_err_f444w', 'isophotal_vegamag_f444w-f470n',
       'isophotal_vegamag_err_f444w-f470n', 'isophotal_vegamag_f770w',
       'isophotal_vegamag_err_f770w', 'isophotal_vegamag_f1130w',
       'isophotal_vegamag_err_f1130w', 'isophotal_vegamag_f1280w',
       'isophotal_vegamag_err_f1280w', 'isophotal_vegamag_f1800w',
       'isophotal_vegamag_err_f1800w', 'Class', 'Prob YSO', 'mag_J', 'e_mag_J',
       'mag_H', 'e_mag_H', 'mag_Ks', 'e_mag_Ks', 'mag_IR1', 'e_mag_IR1',
       'mag_IR2', 'e_mag_IR2', 'mag_IR3', 'e_mag_IR3', 'mag_IR4', 'e_mag_IR4',
       'SPICY_Class_0/1', 'SPICY', 'SPICY_Class'],
      dtype='object')

In [49]:
columns = ['RA', 'DEC', 'size', 'isophotal_vegamag_f090w', 'isophotal_vegamag_f187n',
       'isophotal_vegamag_f200w', 'isophotal_vegamag_f335m', 'isophotal_vegamag_f444w',
       'isophotal_vegamag_f444w-f470n', 'isophotal_vegamag_f770w', 'isophotal_vegamag_f1130w',
       'isophotal_vegamag_f1280w', 'isophotal_vegamag_f1800w', 'Prob YSO','SPICY']
new_cols = ['RA', 'DEC', 'size', 'F090w','F187n','F200w',
       'F335m','F444w','F444w-F470n', 'F770w', 'F1130w', 'F1280w', 'F1800w', 'Prob YSO', 'SPICY']


In [50]:

file_out = file_in.split('.')[0]+'_latex.txt'
print("Printing to ", file_out)
f = open(file_out,'w')

for j in range(0,len(columns)):
    f.write(new_cols[j]+'&')
f.write('\\\ \n')
for i in range(0,len(csv)):
    for j in range(0,len(columns)):
        str_tmp = str("%.4f" %csv[columns[j]][i])+"&"
        f.write(str_tmp)
    f.write('\\\ \n')

f.close()

Printing to  CC_Webb_Predictions_Prob_2_5sigFeb172023_latex.txt


In [51]:
csv_yso = csv[csv.Class==0].reset_index()
csv_yso = csv_yso.iloc[csv_yso.isnull().sum(1).sort_values(ascending=1).index]


file_out = file_in.split('.')[0]+'_latex_ysos.txt'
print("Printing to ", file_out)


f = open(file_out,'w')

for j in range(0,len(columns)):
    f.write(new_cols[j]+'&')
f.write('\\hline \\\ \n')
for i in range(0,len(csv_yso)):
    for j in range(0,len(columns)):
        str_tmp = str("%.4f" %csv_yso[columns[j]].values[i])+"&"
        f.write(str_tmp)
    f.write('\\\ \n')

f.close()

Printing to  CC_Webb_Predictions_Prob_2_5sigFeb172023_latex_ysos.txt
